In [35]:
# Parameters 

application = 'Apple_and_Pears'
varname = 'TMEAN'
target_type = 'cat3'
GCM = 'ECMWF'
# GCM = 'All'
standardized = False
# whether or not to shuffle the training data and the test data, especially recommended if GCM == 'All'
shuffle_train = True 
shuffle_test = True 

In [36]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
%autoreload 2

In [38]:
%matplotlib inline

In [39]:
import sys 
import pathlib

In [40]:
import matplotlib.pyplot as plt 

In [41]:
import numpy as np 
import pandas as pd 
import xarray as xr

In [42]:
np.random.seed(42)

In [43]:
import pycaret

In [44]:
pycaret.__version__

'2.0.0'

In [45]:
from pycaret import datasets

In [46]:
from pycaret.classification import *

In [47]:
df_models = models()

In [48]:
df_models

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors.KNeighborsClassifier,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model.SGDClassifier,True
rbfsvm,SVM - Radial Kernel,sklearn.svm.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process.GPC,False
mlp,MLP Classifier,sklearn.neural_network.MLPClassifier,False
ridge,Ridge Classifier,sklearn.linear_model.RidgeClassifier,True


In [49]:
HOME = pathlib.Path.home()

In [50]:
sys.path.append('/home/nicolasf/research/Smart_Ideas/code/ml4seas')

In [51]:
from evaluation import calc_accuracy_sco
from GCM import prepare_data_CSV_to_CARET

In [52]:
dpath = HOME / 'research' / 'Smart_Ideas' / 'outputs' / 'CSVs'

In [53]:
list(dpath.glob("*.csv"))

[PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_TMEAN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_RAIN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_TMEAN_training_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_std_and_targets_cat3_and_anomalies_RAIN_training_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_RAIN_training_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_TMEAN_training_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_TMEAN_test_set.csv'),
 PosixPath('/home/nicolasf/research/Smart_Ideas/outputs/CSVs/GCMs_and_targets_cat3_and_anomalies_RAIN_test_set.csv'),
 PosixPath('/home/ni

In [56]:
dpath_target = HOME.joinpath(f"research/Smart_Ideas/outputs/targets/application_cases/{application}/SEASONAL/{varname}_N")

In [57]:
[x.name for x in list(dpath_target.glob("*.csv"))]

['Seasonal_TMEAN_N_min_nbdays_over_20C.csv',
 'Seasonal_TMEAN_N_mean_anomalies_and_Q3_categories.csv',
 'Seasonal_TMEAN_N_max_nbdays_over_20C.csv',
 'Seasonal_TMEAN_N_min_nbdays_below_0C.csv',
 'Seasonal_TMEAN_N_max_nbdays_over_25C.csv',
 'Seasonal_TMEAN_N_max_mean.csv',
 'Seasonal_TMEAN_N_min_nbdays_over_25C.csv',
 'Seasonal_TMEAN_N_mean_nbdays_over_25C.csv',
 'Seasonal_TMEAN_N_mean_nbdays_over_30C.csv',
 'Seasonal_TMEAN_N_max_anomalies.csv',
 'Seasonal_TMEAN_N_max_nbdays_over_30C.csv',
 'Seasonal_TMEAN_N_mean_mean.csv',
 'Seasonal_TMEAN_N_max_nbdays_below_0C.csv',
 'Seasonal_TMEAN_N_min_mean.csv',
 'Seasonal_TMEAN_N_min_anomalies.csv',
 'Seasonal_TMEAN_N_mean_nbdays_over_20C.csv',
 'Seasonal_TMEAN_N_mean_anomalies.csv',
 'Seasonal_TMEAN_N_min_nbdays_over_30C.csv',
 'Seasonal_TMEAN_N_mean_nbdays_below_0C.csv']

### get the training and test data 

In [58]:
train_data = pd.read_csv(dpath / f'ECMWF_1981_2010_and_targets_cat3_and_anomalies_{varname}_training_set.csv', index_col=0, parse_dates=True) 
test_data = pd.read_csv(dpath / f'ECMWF_1981_2010_and_targets_cat3_and_anomalies_{varname}_test_set.csv', index_col=0, parse_dates=True)    

In [59]:
train_data.columns

Index(['(-70.0, 70.0)', '(-70.0, 72.5)', '(-70.0, 75.0)', '(-70.0, 77.5)',
       '(-70.0, 80.0)', '(-70.0, 82.5)', '(-70.0, 85.0)', '(-70.0, 87.5)',
       '(-70.0, 90.0)', '(-70.0, 92.5)',
       ...
       'ENI_anoms', 'NSI_anoms', 'WSI_anoms', 'ESI_anoms',
       'NNI_cat3_categories', 'WNI_cat3_categories', 'ENI_cat3_categories',
       'NSI_cat3_categories', 'WSI_cat3_categories', 'ESI_cat3_categories'],
      dtype='object', length=4941)

In [60]:
train_data.iloc[:,-13:].columns

Index(['(60.0, 300.0)', 'NNI_anoms', 'WNI_anoms', 'ENI_anoms', 'NSI_anoms',
       'WSI_anoms', 'ESI_anoms', 'NNI_cat3_categories', 'WNI_cat3_categories',
       'ENI_cat3_categories', 'NSI_cat3_categories', 'WSI_cat3_categories',
       'ESI_cat3_categories'],
      dtype='object')

### only keep the GCM data 

In [61]:
train_data = train_data.iloc[:,:-12]

In [62]:
test_data = test_data.iloc[:,:-12]

### read the target variable 

In [63]:
target = pd.read_csv(dpath_target.joinpath(f"Seasonal_{varname}_N_mean_anomalies_and_Q3_categories.csv"), index_col=0, parse_dates=True) 

In [64]:
target = target.loc[:,[target_type]]

In [65]:
target 

,cat3
time,
1979-03-31,3
1979-04-30,2
1979-05-31,2
1979-06-30,1
1979-07-31,2
...,...
2019-08-31,3
2019-09-30,3
2019-10-31,3


In [66]:
target.loc[train_data.index, :]

,cat3
instance,
1981-03-31,3
1981-03-31,3
1981-03-31,3
1981-03-31,3
1981-03-31,3
...,...
2016-12-31,3
2016-12-31,3
2016-12-31,3


In [67]:
train_data = pd.concat([train_data, target.loc[train_data.index, :]], axis=1)

In [68]:
test_data = pd.concat([test_data, target.loc[test_data.index, :]], axis=1)

In [69]:
train_data.shape

(10750, 4930)

In [70]:
test_data.shape

(36, 4930)

### scale and PCA on the training data 

In [71]:
from sklearn.preprocessing import StandardScaler as scaler 
from sklearn.decomposition import PCA

In [72]:
scaler = scaler()

In [73]:
X = scaler.fit_transform(train_data.iloc[:,:-1])

In [74]:
PCA = PCA(n_components=0.9, random_state=42)

In [75]:
X = PCA.fit_transform(X)

In [76]:
df_train = pd.DataFrame(X, index=train_data.index, columns=[f"PC{i}" for i in range(1, X.shape[1]+1)])

In [77]:
df_train = pd.concat([df_train, train_data.iloc[:,-1]], axis=1)

In [78]:
df_train.columns

Index(['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10',
       'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19',
       'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28',
       'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36', 'PC37',
       'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43', 'PC44', 'PC45', 'PC46',
       'PC47', 'PC48', 'PC49', 'PC50', 'PC51', 'PC52', 'PC53', 'PC54', 'PC55',
       'PC56', 'PC57', 'PC58', 'PC59', 'PC60', 'PC61', 'PC62', 'PC63', 'PC64',
       'PC65', 'PC66', 'PC67', 'PC68', 'PC69', 'PC70', 'PC71', 'PC72', 'PC73',
       'PC74', 'PC75', 'PC76', 'PC77', 'PC78', 'PC79', 'PC80', 'PC81', 'PC82',
       'PC83', 'PC84', 'PC85', 'PC86', 'PC87', 'PC88', 'PC89', 'PC90', 'PC91',
       'PC92', 'PC93', 'PC94', 'PC95', 'PC96', 'PC97', 'PC98', 'PC99', 'cat3'],
      dtype='object')

### then shuffle 

In [79]:
if shuffle_train: 
    df_train = df_train.sample(frac=1., random_state=42, axis=0)

In [80]:
X_test = scaler.transform(test_data.iloc[:,:-1])

In [81]:
X

array([[-15.81790602,  23.58879818, -21.37232258, ...,  -0.91028813,
          0.09891759,  -0.72542959],
       [-22.50962528,  24.65589589, -19.54065923, ...,  -0.27685559,
         -0.71135704,  -4.52021077],
       [-21.99298945,  26.37963782, -12.36650872, ...,  -1.67122143,
         -0.12305885,  -1.95793101],
       ...,
       [ 26.24490631, -30.26935045,  -1.3307667 , ...,  -1.73767768,
          1.0314855 ,   0.20147682],
       [ 28.43174662, -23.3439371 ,  -1.73039832, ...,   0.53654305,
         -0.67428374,  -1.44367676],
       [ 28.49214481, -31.44062109, -14.76809845, ...,   1.29231202,
          2.98943162,  -0.67125228]])

In [82]:
X_test.shape

(36, 4929)

In [83]:
X_test = PCA.transform(X_test)

In [84]:
X_test.shape

(36, 99)

In [85]:
df_test = pd.DataFrame(X_test, index=test_data.index, columns=[f"PC{i}" for i in range(1, X_test.shape[1]+1)])

In [86]:
df_test = pd.concat([df_test, test_data.iloc[:,-1]], axis=1)

In [87]:
df_test.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,cat3
instance,,,,,,,,,,,,,,,,,,,,,
2017-01-31,24.858093,-20.188810,-7.662493,-28.824614,-2.055826,0.867207,17.465190,7.257758,0.239835,22.874083,...,2.995491,0.940122,-1.300863,-1.205838,2.240685,-0.103615,-1.521686,0.901498,-2.322361,3
2017-02-28,27.695154,-22.097676,-3.368357,-16.638057,1.900357,-4.290438,22.710958,2.439020,-11.462522,13.072851,...,0.051176,0.157768,-1.683942,0.299318,1.726721,0.008099,-1.836487,2.683453,-2.084166,3
2017-03-31,28.129596,-16.113271,-10.199480,-2.297828,-0.998691,-8.052899,21.188194,4.464958,-15.779382,8.689024,...,0.377579,1.917749,0.158941,0.143196,2.773346,-1.436346,-1.134381,2.711268,-0.522745,3
2017-04-30,33.646642,-12.881222,-11.841268,-8.021349,-1.801638,-6.154779,18.540891,4.537453,-16.292321,18.815817,...,-1.598405,3.963182,-0.745601,-0.672390,4.509876,-1.421332,-2.013543,0.931097,0.014850,3
2017-05-31,31.895149,-13.926219,-15.309214,-1.618393,6.208229,-16.928059,12.216139,6.428155,-15.780831,17.849558,...,0.455171,2.787512,-0.610274,2.681390,4.399394,-1.314124,-1.613613,1.744191,-0.143554,3


In [88]:
df_train.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC91,PC92,PC93,PC94,PC95,PC96,PC97,PC98,PC99,cat3
instance,,,,,,,,,,,,,,,,,,,,,
1983-05-31,9.424751,44.619015,37.368225,-16.795842,6.517816,-7.559513,-36.753605,-0.761836,5.183112,5.265637,...,1.545996,-3.681314,0.424800,-2.308297,-1.857067,2.113593,-3.179245,2.656842,0.287298,3
2016-02-29,96.138196,-10.277423,12.584620,-16.626916,-3.617158,33.726272,-4.366265,3.670526,17.043464,0.191481,...,4.776645,-1.375387,2.096725,-0.112178,-0.228586,1.064307,-0.645131,-1.617387,2.924973,3
1988-08-31,-9.782165,-7.910161,10.961145,-13.537867,9.212301,9.949728,-14.822843,-1.966379,4.804639,7.379494,...,0.677492,2.832560,-0.349616,1.981176,1.714492,-0.922203,-0.792941,0.274208,-1.601639,3
1998-12-31,-10.476350,-31.511193,29.194351,-5.379314,9.044065,-11.502082,-5.663157,2.365409,-8.275984,3.595966,...,2.759042,0.304985,1.583669,1.225892,2.502253,0.935329,-2.141318,-1.583541,-0.012565,3
1997-03-31,-30.238529,3.752099,-31.000971,13.737943,-8.868457,-5.755460,-20.962406,-5.664135,4.292982,-17.574988,...,0.593063,0.264842,-4.608587,-0.391529,2.769594,-0.060254,-3.167088,0.290064,-0.817790,1


In [89]:
silent = False

In [90]:
log_name = f"{GCM}_1981_2010_pred_{application}_{varname}_{target_type}_target_std_{str(standardized)}_suffle_train_{str(shuffle_train)}"

In [91]:
print(f"experiment on MLFlow in {log_name}")

experiment on MLFlow in ECMWF_1981_2010_pred_Apple_and_Pears_TMEAN_cat3_target_std_False_suffle_train_True


In [92]:
exp_clf = setup(data = df_train, \
                target = f'{target_type}', \
                session_id=123, \
                log_experiment=True, \
                experiment_name=log_name, \
                normalize = True, \
                transformation = False, \
                silent=silent)

Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Multiclass
2,Label Encoded,"1: 0, 2: 1, 3: 2"
3,Original Data,"(10750, 100)"
4,Missing Values,False
5,Numeric Features,99
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [93]:
top5 = compare_models(turbo=True, n_select=5) 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,K Neighbors Classifier,0.9199,0.0000,0.9197,0.9205,0.9199,0.8796,0.8799,0.0490
1,Quadratic Discriminant Analysis,0.8501,0.0000,0.8499,0.8525,0.8507,0.7748,0.7754,0.0801
2,CatBoost Classifier,0.8073,0.0000,0.8045,0.8069,0.8065,0.7098,0.7103,14.2125
3,Extreme Gradient Boosting,0.7764,0.0000,0.7733,0.7757,0.7755,0.6633,0.6638,20.4620
4,Light Gradient Boosting Machine,0.7517,0.0000,0.7479,0.7504,0.7499,0.6258,0.6268,5.7430
5,Extra Trees Classifier,0.7382,0.0000,0.7314,0.7409,0.7333,0.6038,0.6093,0.2568
6,Gradient Boosting Classifier,0.6631,0.0000,0.6576,0.6608,0.6599,0.4913,0.4930,47.2702
7,Naive Bayes,0.6037,0.0000,0.6011,0.6043,0.6036,0.4036,0.4038,0.0090
8,Logistic Regression,0.5998,0.0000,0.5950,0.5949,0.5962,0.3970,0.3978,0.4447
9,Ridge Classifier,0.5984,0.0000,0.5919,0.5896,0.5889,0.3935,0.3970,0.0250


In [94]:
table_top5 = pull()

In [95]:
table_top5

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,K Neighbors Classifier,0.9199,0.0,0.9197,0.9205,0.9199,0.8796,0.8799,0.0490
1,Quadratic Discriminant Analysis,0.8501,0.0,0.8499,0.8525,0.8507,0.7748,0.7754,0.0801
2,CatBoost Classifier,0.8073,0.0,0.8045,0.8069,0.8065,0.7098,0.7103,14.2125
3,Extreme Gradient Boosting,0.7764,0.0,0.7733,0.7757,0.7755,0.6633,0.6638,20.4620
4,Light Gradient Boosting Machine,0.7517,0.0,0.7479,0.7504,0.7499,0.6258,0.6268,5.7430
5,Extra Trees Classifier,0.7382,0.0,0.7314,0.7409,0.7333,0.6038,0.6093,0.2568
6,Gradient Boosting Classifier,0.6631,0.0,0.6576,0.6608,0.6599,0.4913,0.4930,47.2702
7,Naive Bayes,0.6037,0.0,0.6011,0.6043,0.6036,0.4036,0.4038,0.0090
8,Logistic Regression,0.5998,0.0,0.5950,0.5949,0.5962,0.3970,0.3978,0.4447
9,Ridge Classifier,0.5984,0.0,0.5919,0.5896,0.5889,0.3935,0.3970,0.0250


In [96]:
tuned_top5 = [tune_model(i) for i in top5]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8008,0.0000,0.7982,0.8006,0.8004,0.7000,0.7002
1,0.7955,0.0000,0.7925,0.7953,0.7952,0.6920,0.6922
2,0.7729,0.0000,0.7685,0.7713,0.7707,0.6577,0.6590
3,0.8167,0.0000,0.8151,0.8185,0.8172,0.7241,0.7245
4,0.8032,0.0000,0.8003,0.8033,0.8026,0.7034,0.7040
5,0.7633,0.0000,0.7585,0.7604,0.7606,0.6432,0.6443
6,0.7886,0.0000,0.7853,0.7876,0.7879,0.6819,0.6820
7,0.8032,0.0000,0.7997,0.8036,0.8028,0.7034,0.7040
8,0.8165,0.0000,0.8110,0.8159,0.8139,0.7230,0.7251
9,0.8019,0.0000,0.7981,0.8010,0.8002,0.7012,0.7023


In [110]:
tuned_top5

[KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                      metric_params=None, n_jobs=-1, n_neighbors=3, p=2,
                      weights='uniform'),
 QuadraticDiscriminantAnalysis(priors=None, reg_param=0.04,
                               store_covariance=False, tol=0.0001),
 XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
               colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
               importance_type='gain', interaction_constraints='',
               learning_rate=0.18, max_delta_step=0, max_depth=10,
               min_child_weight=2, missing=nan, monotone_constraints='()',
               n_estimators=450, n_jobs=-1, num_class=3, num_parallel_tree=1,
               objective='multi:softprob', random_state=123, reg_alpha=0,
               reg_lambda=1, scale_pos_weight=None, subsample=0.7,
               tree_method='exact', validate_parameters=1, verbosity=0),
 LGBMClassifier(boosting_type='gbdt

In [97]:
def score(model): 
    try: 
        return model.score(df_test.iloc[:,:-1], df_test.iloc[:,-1])
    except: 
        return np.nan

In [98]:
scores = []
for model in tuned_top5: 
    try:
        scores.append(score(model)) 
    except:
        print(f"could not evaluate model {model} on the test set")
        pass

In [101]:
scores

[0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776,
 0.027777777777777776]

In [100]:
tuned_top5_finalized = list(map(finalize_model, tuned_top5))

### blend all models in the libary

Blending models is a method of ensembling which uses consensus among estimators to generate final predictions. The idea behind blending is to combine different machine learning algorithms and use a majority vote or the average predicted probabilities in case of classification to predict the final outcome. Blending models in PyCaret is as simple as writing blend_models. This function can be used to blend specific trained models that can be passed using estimator_list parameter within blend_models or if no list is passed, it will use all the models in model library. In case of Classification, method parameter can be used to define ‘soft‘ or ‘hard‘ where soft uses predicted probabilities for voting and hard uses predicted labels. This functions returns a table with k-fold cross validated scores of common evaluation metrics along with trained model object

In [105]:
blender = blend_models(method='soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8327,0.0000,0.8303,0.8323,0.8318,0.7479,0.7485
1,0.8340,0.0000,0.8317,0.8334,0.8332,0.7500,0.7505
2,0.8234,0.0000,0.8207,0.8227,0.8229,0.7343,0.7344
3,0.8367,0.0000,0.8342,0.8381,0.8367,0.7539,0.7546
4,0.8231,0.0000,0.8200,0.8228,0.8221,0.7334,0.7342
5,0.8165,0.0000,0.8138,0.8148,0.8145,0.7236,0.7245
6,0.8178,0.0000,0.8144,0.8171,0.8165,0.7256,0.7265
7,0.8418,0.0000,0.8387,0.8428,0.8411,0.7613,0.7625
8,0.8338,0.0000,0.8284,0.8334,0.8313,0.7491,0.7513
9,0.8457,0.0000,0.8429,0.8459,0.8456,0.7677,0.7679


In [106]:
table_blender = pull()

In [107]:
table_blender

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8327,0.0,0.8303,0.8323,0.8318,0.7479,0.7485
1,0.8340,0.0,0.8317,0.8334,0.8332,0.7500,0.7505
2,0.8234,0.0,0.8207,0.8227,0.8229,0.7343,0.7344
3,0.8367,0.0,0.8342,0.8381,0.8367,0.7539,0.7546
4,0.8231,0.0,0.8200,0.8228,0.8221,0.7334,0.7342
5,0.8165,0.0,0.8138,0.8148,0.8145,0.7236,0.7245
6,0.8178,0.0,0.8144,0.8171,0.8165,0.7256,0.7265
7,0.8418,0.0,0.8387,0.8428,0.8411,0.7613,0.7625
8,0.8338,0.0,0.8284,0.8334,0.8313,0.7491,0.7513
9,0.8457,0.0,0.8429,0.8459,0.8456,0.7677,0.7679


### score the blender on the test data 

In [108]:
score(blender)

0.027777777777777776

------

### Now re-tune the best model 

In [113]:
best_model = tune_model(top5[0], choose_better=True)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9296,0.0000,0.9293,0.9302,0.9296,0.8942,0.8945
1,0.9309,0.0000,0.9320,0.9319,0.9310,0.8963,0.8967
2,0.9429,0.0000,0.9432,0.9435,0.9430,0.9142,0.9145
3,0.9456,0.0000,0.9451,0.9457,0.9455,0.9181,0.9183
4,0.9441,0.0000,0.9433,0.9441,0.9439,0.9160,0.9162
5,0.9415,0.0000,0.9407,0.9417,0.9416,0.9120,0.9121
6,0.9495,0.0000,0.9497,0.9496,0.9495,0.9240,0.9241
7,0.9481,0.0000,0.9478,0.9482,0.9480,0.9220,0.9221
8,0.9495,0.0000,0.9491,0.9496,0.9494,0.9240,0.9241
9,0.9521,0.0000,0.9526,0.9526,0.9522,0.9281,0.9283


In [114]:
table_best_model = pull()

### score over the test set 

In [115]:
score(best_model)

0.027777777777777776

### retrain the model over the whole training set 

In [116]:
retrained_best_model = finalize_model(best_model)

In [117]:
score(retrained_best_model)

0.027777777777777776

### Now save the notebook in HTML and rename to reflect the parameters of the experiment 

In [62]:
notebook_name='Apple_and_Pears.ipynb'

In [63]:
!jupyter nbconvert --to html {notebook_name}

[NbConvertApp] Converting notebook pycaret_v2_wPCA.ipynb to html
[NbConvertApp] Writing 432145 bytes to pycaret_v2_wPCA.html


In [64]:
savepath = pathlib.Path('./saved_notebooks/')

In [65]:
if not savepath.exists(): 
    savepath.mkdir(parents=True)

In [66]:
output_name = savepath.joinpath(f"pycaret_classification_{log_name}.html")

In [67]:
pathlib.Path('./pycaret_v2_wPCA.html').rename(output_name)

PosixPath('saved_notebooks/pycaret_classification_ECMWF_pred_ESI_TMEAN_cat3_categories_target_std_False_suffle_train_True.html')